## Задание 1  
> Подберите скорость обучения (alpha) и количество итераций



In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#функция генератор исходных данных
def gen_data(N_features,N_samples,X_range,Noise_sigma,flag=True):
    if flag: np.random.seed(100)
    n_features=N_features # число признакиов
    n_samples = N_samples # число объктов 
    x_range=X_range 
    noise_sigma=Noise_sigma # объекты
    w_true = np.random.normal(size=(n_features,)) #истинные коэффициенты
    X = np.random.uniform(-x_range,x_range, (n_samples,n_features)) #признаки
    #print(w_true.shape)
    Y= X.dot(w_true)+np.random.normal(0, noise_sigma, size=(n_samples)) # объекты
    return [X,Y,w_true]
 
    
    
    
    

In [3]:
#функция ошибок
def mse_error(Y_cur,Y):
    return (np.sum((Y - Y_cur)**2))/len(Y)

аналитическое решение, минимизирующее функцию ошибок определяется выражением  $\omega=(𝑋^𝑇𝑋)^{−1}𝑋^𝑇y$   &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;(1)  
Код на python приведен ниже

In [4]:
def theoretic_solution(X,y):
    #linalg.solve ищет корни линейного уравнения (𝑋^𝑇*𝑋)w = 𝑋^𝑇*Y. Домножили левую и правую часть (1) на 𝑋^𝑇*𝑋.
    return np.linalg.solve(X.T.dot(X), X.T.dot(Y))
    

In [5]:
n_features=2
n_samples = 1000
x_range=10
noise_sigma=0.5
np.random.seed(100)
X,Y,w_true = gen_data(n_features,n_samples,x_range,noise_sigma) # массив признаков, ответа и истинных коэффициентов
w_theor = theoretic_solution(X,Y) # аналитическое решение
Y_cur  = X.dot(w_theor)
print(f'Среднеквадратическая ошибка линейной регрессии по рассчитанным аналитически коэффициентам равна {mse_error(Y_cur,Y)}')

Среднеквадратическая ошибка линейной регрессии по рассчитанным аналитически коэффициентам равна 0.24929383411249378


Определение коэффициентов линейной регрессии методом градиентного спуска  поизводится итерационно посредством пошагового выполнения следующей операции: $ \omega_i = \omega_{i-1} - \alpha \nabla Q(\omega_{i-1}, X) $,  
где $ \nabla Q(\omega_{i-1}, X) = \frac{2}{l}X^T(X\omega - y) $.  
Ниже приведена реализация на python.


In [19]:
w = np.zeros(n_features) # начальная точка для поиска минимума
max_step = 1e5 #максимальное число шагов
min_dist = 1e-5 # минимальное (стремимся) расстояние между соседними весами по норме L2
alpha = 0.011 #шаг градиента
w_list = [w.copy()] #  список коэффициентов
err_list = [] #список с ошибками
cur_error = np.inf #текущая ошибка линейной регрессии
cur_step=0
weight_dist = np.inf

while weight_dist > min_dist and cur_step < max_step:
    new_w = w - 2* alpha *  np.dot(X.T, (np.dot(X, w) - Y)) / Y.shape[0]
    weight_dist = np.linalg.norm(new_w-w, ord = 2)
    cur_step+=1
    w_list.append(new_w.copy())
    Y_cur =  X.dot(new_w )
    err_list.append(mse_error(Y_cur,Y))
    w = new_w
print(f'"Истинные" коэффициенты линейной регрессии => {w_true}')    
print(f'Расчитанные "аналитически" коэффициенты линейной регрессии => {w_theor}, функционал ошибки => {round(mse_error(X.dot(w_theor),Y),5)} ')
print(f'Расчитанные методом градиентного спуска коэффициенты линейной регрессии => {w}, функционал ошибки => {round(mse_error(X.dot(w),Y),5)} ')
print(f'Подгонка коэффициентов произошла за {cur_step} шагов, шаг градиента => {alpha}')

"Истинные" коэффициенты линейной регрессии => [-1.74976547  0.3426804 ]
Расчитанные "аналитически" коэффициенты линейной регрессии => [-1.75186518  0.3439976 ], функционал ошибки => 0.24929 
Расчитанные методом градиентного спуска коэффициенты линейной регрессии => [-1.75186233  0.34399787], функционал ошибки => 0.24929 
Подгонка коэффициентов произошла за 10 шагов, шаг градиента => 0.011


Задавшись минимальным расстоянием сежду весами была подобрана оптимальная скорость обучения (0.011), обеспечивающая заданную точность за 10 шагов.

## Задание 2  
>В коде ниже коде нет итерации по весам, но здесь есть ошибка. Необходимо исправить ее.

In [7]:
n = X.shape[1]
alpha = 1e-2
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')
print(X.shape, W.shape,Y.shape)

for i in range(100):
    y_pred = np.dot(X, W)
    err = mse_error(y_pred, Y)
    #print(Y.shape, y_pred.shape, X.shape)
    # W -= alpha * (1/n * 2 * np.sum(X * (y_pred - y))) #ОШИБОЧНАЯ СТРОКА
    W -= alpha * (1/X.shape[0] * 2 * np.sum(X.T * (y_pred - Y), axis = 1))
    W_pred = W
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')
        



Number of objects = 2        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

(1000, 2) (2,) (1000,)
Iteration #0: W_new = [-0.83934085  0.41754989], MSE = 253.97
Iteration #10: W_new = [-1.75185005  0.34400271], MSE = 0.25
Iteration #20: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #30: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #40: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #50: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #60: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #70: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #80: W_new = [-1.75186518  0.3439976 ], MSE = 0.25
Iteration #90: W_new = [-1.75186518  0.3439976 ], MSE = 0.25


### Строка с ошибкой закоментирована и заменена верной.  
В первоначальном варианте суммирование записано по неправильной оси и нормировка производится  на число признаков, а не на число объектов. Кроме того массив X надо транспонировать. В противном случае по правилам транслитерации при обычном умножении матриц возникает ошибка.

## Задание 3  
>Вместо того чтобы задавать количество итераций, задайте условие остановки алгоритма, когда ошибка за итерацию начинает изменяться ниже определённого порога — упрощённый аналог параметра tol в линейной регрессии в sklearn.

In [13]:

n = X.shape[1]
alpha = 1e-2
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')
print(X.shape, W.shape,Y.shape)
delta__err_max = 1e-5 # минимальное приращение ошибки MSE, по которому происходит завершение
delta__err_cur  = np.inf
err_pred = np.inf
i=0
while delta__err_cur >= delta__err_max:
    y_pred = np.dot(X, W)
    err = mse_error(y_pred, Y)
    #print(Y.shape, y_pred.shape, X.shape)
    # W -= alpha * (1/n * 2 * np.sum(X * (y_pred - y))) #ОШИБОЧНАЯ СТРОКА    
    W -= alpha * (1/X.shape[0] * 2 * np.sum(X.T * (y_pred - Y), axis = 1))
    delta__err_cur = np.abs(err - err_pred)
    err_pred = err
    i+=1
print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 2        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

(1000, 2) (2,) (1000,)
Iteration #10: W_new = [-1.7518198  0.3440117], MSE = 0.25
